# AKS 클러스터 실습 가이드

> **실습 환경**: 이 노트북은 GitHub Codespaces에서 실행하도록 구성되어 있습니다. Azure CLI, kubectl 등 필요한 도구가 미리 설치되어 있습니다.

> **사전 준비**: Azure 구독과 이전 실습에서 생성한 ACR이 필요합니다.

이 노트북에서는 Azure Kubernetes Service(AKS) 클러스터 생성, 쿠버네티스 및 AKS의 기본 개념, 컨테이너 이미지 배포, 실행, 확장, 모니터링 실습을 진행합니다.

## 1. 쿠버네티스와 AKS 기본 개념 및 이점

### 쿠버네티스란?
쿠버네티스(Kubernetes)는 컨테이너화된 애플리케이션의 배포, 확장, 관리를 자동화하는 오픈소스 플랫폼입니다.

#### 쿠버네티스의 주요 이점
- **자동화된 배포 및 확장**
- **자가 치유(Self-healing)**
- **서비스 디스커버리 및 로드밸런싱**
- **롤링 업데이트 및 롤백**

### AKS란?
AKS(Azure Kubernetes Service)는 Azure에서 제공하는 관리형 쿠버네티스 서비스로, 클러스터 관리의 복잡성을 줄여줍니다.

#### AKS의 주요 이점
- **간편한 클러스터 관리**
- **Azure와의 통합**
- **자동 업그레이드 및 패치**

## 2. AKS 클러스터 생성 실습

Azure CLI를 사용하여 AKS 클러스터를 생성합니다.

> **GitHub Codespaces**: Azure CLI가 이미 설치되어 있으므로 바로 사용 가능합니다.

### 2.1 Azure 로그인

먼저 Azure에 로그인합니다.

In [3]:
%%bash
# Azure 로그인 세션 확인
if az account show &>/dev/null; then
  echo "✅ 이미 Azure에 로그인되어 있습니다."
  echo "   현재 구독: $(az account show --query name -o tsv)"
else
  echo "🔐 Azure 로그인이 필요합니다. 로그인을 진행합니다..."
  az login --output none
  
  # 첫 번째 구독을 자동으로 선택
  SUBSCRIPTION_ID=$(az account list --query "[0].id" -o tsv)
  az account set --subscription $SUBSCRIPTION_ID
  
  echo "✅ Azure 로그인 완료"
  echo "   사용 중인 구독: $(az account show --query name -o tsv)"
fi

✅ 이미 Azure에 로그인되어 있습니다.


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


   현재 구독: Visual Studio Enterprise Subscription


### 2.2 공통 변수 설정

1번 노트북에서 생성한 리소스 그룹을 사용하며, AKS 클러스터 이름을 설정합니다.

> **중요**: 이 셀에서 설정한 변수들은 이후 모든 셀에서 공통으로 사용됩니다. 리소스 이름을 변경하려면 이 셀만 수정하면 됩니다.

In [9]:
import subprocess

# 공통 변수 설정 (한 곳에서만 수정하면 전체 적용)
RESOURCE_GROUP = "myResourceGroup"
AKS_CLUSTER_NAME = "aks-mini-labs-cluster"

print(f"📝 설정된 변수:")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   AKS 클러스터: {AKS_CLUSTER_NAME}")
print()

# 리소스 그룹 존재 확인
result = subprocess.run(
    ["az", "group", "show", "--name", RESOURCE_GROUP],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(f"✅ 리소스 그룹이 존재합니다: {RESOURCE_GROUP}")
    subprocess.run(
        ["az", "group", "show", "--name", RESOURCE_GROUP, 
         "--query", "{Name:name, Location:location}", "--output", "table"]
    )
else:
    print(f"❌ 리소스 그룹이 존재하지 않습니다: {RESOURCE_GROUP}")
    print("   1번 노트북을 먼저 실행해주세요.")

📝 설정된 변수:
   리소스 그룹: myResourceGroup
   AKS 클러스터: aks-mini-labs-cluster

✅ 리소스 그룹이 존재합니다: myResourceGroup
✅ 리소스 그룹이 존재합니다: myResourceGroup


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name             Location
---------------  ------------
myResourceGroup  koreacentral


### 2.3 AKS 클러스터 생성 (관리형 Prometheus 및 Grafana 활성화)

AKS 클러스터 생성 시 Azure Monitor의 관리형 Prometheus와 Grafana를 함께 활성화합니다.

> **참고**: 클러스터 생성에는 약 5-10분 정도 소요됩니다.

In [6]:
import subprocess

# 위 셀에서 설정한 변수 사용
print("🚀 AKS 클러스터 생성을 시작합니다...")
print("⏱️  예상 소요 시간: 5-10분")
print()
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   클러스터 이름: {AKS_CLUSTER_NAME}")
print()

# 관리형 Prometheus 및 Grafana가 활성화된 AKS 클러스터 생성
result = subprocess.run([
    "az", "aks", "create",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--node-count", "2",
    "--enable-addons", "monitoring",
    "--enable-managed-identity",
    "--generate-ssh-keys",
    "--verbose"
])

if result.returncode == 0:
    print()
    print("✅ 클러스터 생성 완료!")
    print()
    
    # 클러스터 생성 상태 확인
    print("📋 클러스터 목록:")
    subprocess.run(["az", "aks", "list", "--resource-group", RESOURCE_GROUP, "--output", "table"])
    
    print()
    print("📊 클러스터 상세 정보:")
    subprocess.run(["az", "aks", "show", "--resource-group", RESOURCE_GROUP, 
                    "--name", AKS_CLUSTER_NAME, "--output", "table"])
else:
    print("❌ 클러스터 생성 실패")

🚀 AKS 클러스터 생성을 시작합니다...
⏱️  예상 소요 시간: 5-10분



/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
INFO: Use existing SSH public key file: /Users/junwoojeong/.ssh/id_rsa.pub
INFO: Request URL: 'https://management.azure.com//subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/locations?api-version=2019-11-01'
INFO: Request method: 'GET'
INFO: Request headers:
INFO:     'User-Agent': 'python/3.12.12 (macOS-26.0.1-arm64-arm-64bit) AZURECLI/2.77.0 (PIP)'
INFO:     'Accept-Encoding': 'gzip, deflate'
INFO:     'Accept': '*/*'
INFO:     'Connection': 'keep-alive'
INFO:     'x-ms-client-request-id': '14856750-680f-4e54-9861-9de429cffbba'
INFO:     'CommandName': 'aks create'
INFO:     'ParameterSetName': '--resource-group --name -

{
  "aadProfile": null,
  "addonProfiles": {
    "omsagent": {
      "config": {
        "logAnalyticsWorkspaceResourceID": "/subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/DefaultResourceGroup-SE/providers/Microsoft.OperationalInsights/workspaces/DefaultWorkspace-8627ae60-01d3-4a2d-9c33-89dea54cd4b4-SE",
        "useAADAuth": "true"
      },
      "enabled": true,
      "identity": null
    }
  },
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "capacityReservationGroupId": null,
      "count": 2,
      "creationData": null,
      "currentOrchestratorVersion": "1.32.7",
      "eTag": null,
      "enableAutoScaling": false,
      "enableEncryptionAtHost": false,
      "enableFips": false,
      "enableNodePublicIp": false,
      "enableUltraSsd": false,
      "gatewayProfile": null,
      "gpuInstanceProfile": null,
      "gpuProfile": null,
      "hostGroupId": null,
      "kubeletConfig": null,
      "kubeletDiskType": "OS",
      "linuxOsConf

INFO: Command ran in 266.581 seconds (init: 0.820, invoke: 265.761)



✅ 클러스터 생성 완료!

📋 클러스터 목록:


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name                   Location      ResourceGroup    KubernetesVersion    CurrentKubernetesVersion    ProvisioningState    Fqdn
---------------------  ------------  ---------------  -------------------  --------------------------  -------------------  ---------------------------------------------------------------------
aks-mini-labs-cluster  koreacentral  myResourceGroup  1.32                 1.32.7                      Succeeded            aks-mini-l-myresourcegroup-8627ae-a07v024w.hcp.koreacentral.azmk8s.io

📊 클러스터 상세 정보:


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name                   Location      ResourceGroup    KubernetesVersion    CurrentKubernetesVersion    ProvisioningState    Fqdn
---------------------  ------------  ---------------  -------------------  --------------------------  -------------------  ---------------------------------------------------------------------
aks-mini-labs-cluster  koreacentral  myResourceGroup  1.32                 1.32.7                      Succeeded            aks-mini-l-myresourcegroup-8627ae-a07v024w.hcp.koreacentral.azmk8s.io
   ProvisioningState    Fqdn
---------------------  ------------  ---------------  -------------------  --------------------------  -------------------  ---------------------------------------------------------------------
aks-mini-labs-cluster  koreacentral  myResourceGroup  1.32                 1.32.7                      Succeeded            aks-mini-l-myresourcegroup-8627ae-a07v024w.hcp.koreacentral.azmk8s.io


### 2.4 클러스터 인증 정보 가져오기

kubectl을 사용하기 위해 AKS 클러스터의 인증 정보를 가져옵니다.

> **GitHub Codespaces**: kubectl이 이미 설치되어 있습니다.

In [30]:
import subprocess

# 위 셀에서 설정한 변수 사용
print(f"🔑 클러스터 인증 정보 가져오기: {AKS_CLUSTER_NAME}")
print()

# 클러스터 인증 정보 가져오기
subprocess.run(["az", "aks", "get-credentials", 
                "--resource-group", RESOURCE_GROUP, 
                "--name", AKS_CLUSTER_NAME])

print()
print("📊 kubectl 연결 확인:")
subprocess.run(["kubectl", "get", "nodes"])

print()
print("ℹ️ 클러스터 정보:")
subprocess.run(["kubectl", "cluster-info"])

🔑 클러스터 인증 정보 가져오기: aks-mini-labs-cluster



/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version



📊 kubectl 연결 확인:
NAME                                STATUS   ROLES    AGE   VERSION
aks-nodepool1-39887763-vmss000000   Ready    <none>   89m   v1.32.7
aks-nodepool1-39887763-vmss000001   Ready    <none>   89m   v1.32.7

ℹ️ 클러스터 정보:
Kubernetes control plane is running at https://aks-mini-l-myresourcegroup-8627ae-a07v024w.hcp.koreacentral.azmk8s.io:443
CoreDNS is running at https://aks-mini-l-myresourcegroup-8627ae-a07v024w.hcp.koreacentral.azmk8s.io:443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://aks-mini-l-myresourcegroup-8627ae-a07v024w.hcp.koreacentral.azmk8s.io:443/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


CompletedProcess(args=['kubectl', 'cluster-info'], returncode=0)

## 3. 컨테이너 이미지 AKS에 배포하기

앞서 ACR에 푸시한 이미지를 AKS에 배포합니다.

### 3.1 ACR을 AKS에 연결

AKS 클러스터가 ACR에 접근할 수 있도록 권한을 부여합니다.

In [31]:
import subprocess

# 위 셀에서 설정한 변수 사용
# ACR 이름을 리소스 그룹에서 자동으로 가져오기
print("🔍 ACR 정보 확인 중...")
acr_result = subprocess.run(
    ["az", "acr", "list", "--resource-group", RESOURCE_GROUP, "--query", "[0].name", "-o", "tsv"],
    capture_output=True,
    text=True
)

if acr_result.returncode == 0 and acr_result.stdout.strip():
    ACR_NAME = acr_result.stdout.strip()
    print(f"✅ ACR 발견: {ACR_NAME}")
else:
    print("❌ ACR을 찾을 수 없습니다. 1번 노트북을 먼저 실행해주세요.")
    raise Exception("ACR not found")

print()
print(f"🔗 ACR을 AKS에 연결 중...")
print(f"   ACR: {ACR_NAME}")
print(f"   AKS: {AKS_CLUSTER_NAME}")
print()

# ACR을 AKS에 연결 (attach-acr)
# 이 명령이 자동으로 AcrPull 권한을 kubelet identity에 부여합니다
result = subprocess.run(["az", "aks", "update", 
                "--resource-group", RESOURCE_GROUP, 
                "--name", AKS_CLUSTER_NAME, 
                "--attach-acr", ACR_NAME])

if result.returncode == 0:
    print()
    print("✅ ACR 연결 완료!")
    print("   AKS가 ACR에서 이미지를 pull할 수 있는 권한이 자동으로 설정되었습니다.")
else:
    print()
    print("❌ ACR 연결 실패")

🔍 ACR 정보 확인 중...
✅ ACR 발견: myacr1760169422

🔗 ACR을 AKS에 연결 중...
   ACR: myacr1760169422
   AKS: aks-mini-labs-cluster

✅ ACR 발견: myacr1760169422

🔗 ACR을 AKS에 연결 중...
   ACR: myacr1760169422
   AKS: aks-mini-labs-cluster



/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
AAD role propagation done[############################################]  100.0000%WARNING: Could not find service principal or user assigned MSI for roleassignment
AAD role propagation done[############################################]  100.0000%

{
  "aadProfile": null,
  "addonProfiles": {
    "azurepolicy": {
      "config": null,
      "enabled": true,
      "identity": {
        "clientId": "fe5e4df5-1c69-4567-90ba-2d89ad0b6d80",
        "objectId": "ffa53af1-d1ef-4bb9-bfca-6c5a9fa07e7b",
        "resourceId": "/subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourcegroups/MC_myResourceGroup_aks-mini-labs-cluster_koreacentral/providers/Microsoft.ManagedIdentity/userAssignedIdentities/azurepolicy-aks-mini-labs-cluster"
      }
    },
    "omsagent": {
      "config": {
        "logAnalyticsWorkspaceResourceID": "/subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/DefaultResourceGroup-SE/providers/Microsoft.OperationalInsights/workspaces/DefaultWorkspace-8627ae60-01d3-4a2d-9c33-89dea54cd4b4-SE",
        "useAADAuth": "true"
      },
      "enabled": true,
      "identity": null
    }
  },
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "capacityReservationGroupId": null,
      "count":

### 3.2 ACR 이미지 확인

배포하기 전에 ACR에 어떤 이미지가 있는지 확인합니다.

In [35]:
import subprocess

print(f"📦 ACR 저장소 목록 확인: {ACR_NAME}")
print()

# ACR 저장소 목록 조회
repo_result = subprocess.run(
    ["az", "acr", "repository", "list", "--name", ACR_NAME, "--output", "table"],
    capture_output=True,
    text=True
)

if repo_result.returncode == 0:
    print(repo_result.stdout)
    
    # 저장소 이름 추출하여 태그 확인
    repos_output = subprocess.run(
        ["az", "acr", "repository", "list", "--name", ACR_NAME, "-o", "tsv"],
        capture_output=True,
        text=True
    )
    
    if repos_output.returncode == 0 and repos_output.stdout.strip():
        repos = repos_output.stdout.strip().split('\n')
        print()
        print("🏷️  각 이미지의 태그 확인:")
        print()
        
        for repo in repos:
            if repo:
                print(f"   📂 저장소: {repo}")
                subprocess.run([
                    "az", "acr", "repository", "show-tags",
                    "--name", ACR_NAME,
                    "--repository", repo,
                    "--output", "table"
                ])
                print()
else:
    print("❌ ACR 저장소 목록 조회 실패")
    print(repo_result.stderr)

print()
print("💡 1번 노트북에서 이미지를 푸시했는지 확인하세요.")
print("   이미지가 없다면 1번 노트북의 섹션 6.7(이미지 푸시)를 실행하세요.")

📦 ACR 저장소 목록 확인: myacr1760169422

Result
--------
myapp


🏷️  각 이미지의 태그 확인:

   📂 저장소: myapp


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Result
--------
latest


💡 1번 노트북에서 이미지를 푸시했는지 확인하세요.
   이미지가 없다면 1번 노트북의 섹션 6.7(이미지 푸시)를 실행하세요.


### 3.3 Deployment YAML 파일 작성

`deployment.yaml` 파일을 자동으로 생성합니다.

In [32]:
# 위 셀에서 정의한 ACR_NAME 변수 사용하여 deployment.yaml 생성
deployment_yaml = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: myapp
spec:
  replicas: 2
  selector:
    matchLabels:
      app: myapp
  template:
    metadata:
      labels:
        app: myapp
    spec:
      containers:
      - name: myapp
        image: {ACR_NAME}.azurecr.io/myapp:latest
        ports:
        - containerPort: 8080
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi
"""

# 파일 생성
with open("deployment.yaml", "w") as f:
    f.write(deployment_yaml)

print(f"✅ deployment.yaml 파일 생성 완료")
print(f"   이미지: {ACR_NAME}.azurecr.io/myapp:latest")
print()
print("📄 생성된 파일 내용:")
print(deployment_yaml)

✅ deployment.yaml 파일 생성 완료
   이미지: myacr1760169422.azurecr.io/myapp:latest

📄 생성된 파일 내용:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: myapp
spec:
  replicas: 2
  selector:
    matchLabels:
      app: myapp
  template:
    metadata:
      labels:
        app: myapp
    spec:
      containers:
      - name: myapp
        image: myacr1760169422.azurecr.io/myapp:latest
        ports:
        - containerPort: 8080
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi



### 3.4 Deployment 생성 및 Service 노출

In [33]:
import subprocess

print("📦 Deployment 생성 중...")
subprocess.run(["kubectl", "apply", "-f", "deployment.yaml"])

print()
print("📊 Deployment 상태 확인:")
subprocess.run(["kubectl", "get", "deployments"])

print()
print("🔍 Pod 상태 확인:")
subprocess.run(["kubectl", "get", "pods"])

print()
print("🌐 Service 생성 중 (LoadBalancer)...")
subprocess.run(["kubectl", "expose", "deployment", "myapp", 
                "--type=LoadBalancer", 
                "--port=80", 
                "--target-port=8080"])

print()
print("📋 Service 상태 확인:")
subprocess.run(["kubectl", "get", "services"])

📦 Deployment 생성 중...
deployment.apps/myapp unchanged

📊 Deployment 상태 확인:
NAME    READY   UP-TO-DATE   AVAILABLE   AGE
myapp   0/2     1            0           69m

🔍 Pod 상태 확인:
deployment.apps/myapp unchanged

📊 Deployment 상태 확인:
NAME    READY   UP-TO-DATE   AVAILABLE   AGE
myapp   0/2     1            0           69m

🔍 Pod 상태 확인:
NAME                     READY   STATUS             RESTARTS   AGE
myapp-766c44fd57-n8qlm   0/1     ImagePullBackOff   0          53m
myapp-c86f859c8-chxv4    0/1     ErrImagePull       0          62m
myapp-c86f859c8-vwz97    0/1     ErrImagePull       0          62m

🌐 Service 생성 중 (LoadBalancer)...
NAME                     READY   STATUS             RESTARTS   AGE
myapp-766c44fd57-n8qlm   0/1     ImagePullBackOff   0          53m
myapp-c86f859c8-chxv4    0/1     ErrImagePull       0          62m
myapp-c86f859c8-vwz97    0/1     ErrImagePull       0          62m

🌐 Service 생성 중 (LoadBalancer)...

📋 Service 상태 확인:
NAME         TYPE           CLUSTER-IP     

Error from server (AlreadyExists): services "myapp" already exists


CompletedProcess(args=['kubectl', 'get', 'services'], returncode=0)

### 3.5 외부 IP 확인 및 애플리케이션 접근
print()

subprocess.run(["kubectl", "get", "service", "myapp"])

print()
print("💡 외부 IP가 '<pending>'이면 잠시 후 다시 실행하세요.")
print("   외부 IP가 할당되면 해당 IP로 브라우저나 curl로 접근 가능합니다.")
print("   예: curl http://<EXTERNAL-IP>")

## 4. 애플리케이션 확장

Kubernetes에서는 replica 수를 조정하여 애플리케이션을 쉽게 확장할 수 있습니다.

### 4.1 수동 스케일링

In [37]:
import subprocess

print("📈 Replica 수를 4로 확장 중...")
subprocess.run(["kubectl", "scale", "deployment", "myapp", "--replicas=4"])

print()
print("🔍 Pod 상태 확인 (상세):")
subprocess.run(["kubectl", "get", "pods", "-o", "wide"])

print()
print("📊 Deployment 상태:")
subprocess.run(["kubectl", "get", "deployment", "myapp"])

📈 Replica 수를 4로 확장 중...
deployment.apps/myapp scaled

🔍 Pod 상태 확인 (상세):
NAME                     READY   STATUS              RESTARTS   AGE   IP             NODE                                NOMINATED NODE   READINESS GATES
myapp-766c44fd57-pftnj   1/1     Running             0          86s   10.244.1.12    aks-nodepool1-39887763-vmss000001   <none>           <none>
myapp-766c44fd57-tpl7s   1/1     Running             0          82s   10.244.0.115   aks-nodepool1-39887763-vmss000000   <none>           <none>
myapp-766c44fd57-vcg9n   0/1     ContainerCreating   0          0s    <none>         aks-nodepool1-39887763-vmss000001   <none>           <none>

📊 Deployment 상태:
deployment.apps/myapp scaled

🔍 Pod 상태 확인 (상세):
NAME                     READY   STATUS              RESTARTS   AGE   IP             NODE                                NOMINATED NODE   READINESS GATES
myapp-766c44fd57-pftnj   1/1     Running             0          86s   10.244.1.12    aks-nodepool1-39887763-vmss000001 

CompletedProcess(args=['kubectl', 'get', 'deployment', 'myapp'], returncode=0)

### 4.2 HPA(Horizontal Pod Autoscaler)로 오토스케일링 적용

HPA를 사용하면 CPU 사용률 등의 메트릭에 따라 자동으로 Pod 수를 조정할 수 있습니다.

In [38]:
import subprocess

print("🎯 HPA 생성 중 (CPU 사용률 50% 기준, 최소 2개, 최대 10개)...")
subprocess.run(["kubectl", "autoscale", "deployment", "myapp", 
                "--cpu-percent=50", 
                "--min=2", 
                "--max=10"])

print()
print("📊 HPA 상태:")
subprocess.run(["kubectl", "get", "hpa"])

print()
print("📋 HPA 상세 정보:")
subprocess.run(["kubectl", "describe", "hpa", "myapp"])

🎯 HPA 생성 중 (CPU 사용률 50% 기준, 최소 2개, 최대 10개)...
horizontalpodautoscaler.autoscaling/myapp autoscaled

📊 HPA 상태:
NAME    REFERENCE          TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
myapp   Deployment/myapp   cpu: <unknown>/50%   2         10        0          0s

📋 HPA 상세 정보:
horizontalpodautoscaler.autoscaling/myapp autoscaled

📊 HPA 상태:
NAME    REFERENCE          TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
myapp   Deployment/myapp   cpu: <unknown>/50%   2         10        0          0s

📋 HPA 상세 정보:
Name:                                                  myapp
Namespace:                                             default
Labels:                                                <none>
Annotations:                                           <none>
CreationTimestamp:                                     Sat, 11 Oct 2025 19:12:06 +0900
Reference:                                             Deployment/myapp
Metrics:                                               ( curr

CompletedProcess(args=['kubectl', 'describe', 'hpa', 'myapp'], returncode=0)

### 4.3 HPA 오토스케일링 테스트

HPA가 정상적으로 동작하는지 확인하기 위해 부하를 발생시킵니다.

#### 방법 1: 부하 생성 도구 사용

In [39]:
import subprocess

print("🔥 부하 생성용 Pod 실행 중...")
subprocess.run(["kubectl", "run", "load-generator", 
                "--image=busybox", 
                "--restart=Never", 
                "--", "/bin/sh", "-c", 
                "while true; do wget -q -O- http://myapp; done"])

print()
print("✅ 부하 생성 Pod가 실행되었습니다.")
print()
print("   HPA 상태를 모니터링하려면 별도 터미널에서 다음 명령 실행:")
print("   kubectl get hpa myapp --watch")
print()
print("   또는 Pod 수 변화 모니터링:")
print("   kubectl get pods --watch")

🔥 부하 생성용 Pod 실행 중...
pod/load-generator created

✅ 부하 생성 Pod가 실행되었습니다.

   HPA 상태를 모니터링하려면 별도 터미널에서 다음 명령 실행:
   kubectl get hpa myapp --watch

   또는 Pod 수 변화 모니터링:
   kubectl get pods --watch


#### 테스트 정리

부하 테스트가 완료되면 부하 생성 Pod를 삭제합니다.

In [40]:
import subprocess

print("🧹 부하 생성 Pod 삭제 중...")
subprocess.run(["kubectl", "delete", "pod", "load-generator"])

print()
print("✅ 부하 생성 Pod가 삭제되었습니다.")

🧹 부하 생성 Pod 삭제 중...
pod "load-generator" deleted

✅ 부하 생성 Pod가 삭제되었습니다.


### HPA 오토스케일링 테스트 방법
HPA가 정상적으로 동작하는지 확인하려면, 파드에 부하를 인위적으로 발생시켜 오토스케일링이 동작하는지 관찰할 수 있습니다.

1. **stress 도구 설치** (app 파드에 설치):
```bash
kubectl exec -it <app-pod-name> -- apt-get update
kubectl exec -it <app-pod-name> -- apt-get install -y stress
```

2. **CPU 부하 발생**:
```bash
kubectl exec -it <app-pod-name> -- stress --cpu 1 --timeout 300
```

3. **HPA 상태 모니터링**:
```bash
kubectl get hpa -w
```

- 위 과정을 통해 CPU 사용률이 증가하면 HPA가 자동으로 파드 수를 늘리는 것을 확인할 수 있습니다.

 > 참고: `<app-pod-name>`은 실제 app 파드 이름으로 대체하세요. 파드 이름은 `kubectl get pods` 명령으로 확인할 수 있습니다.

## 5. 모니터링

AKS는 Azure Monitor, Prometheus, Grafana와 통합되어 클러스터 및 애플리케이션 상태를 모니터링할 수 있습니다.

### 5.1 Azure Monitor 기본 모니터링

Azure Portal에서 AKS 리소스를 선택하고, 모니터링 탭에서 로그와 메트릭을 확인하세요.

### 5.2 관리형 Prometheus 및 Grafana 활성화

AKS 클러스터에 Azure Monitor 관리형 Prometheus와 Grafana를 활성화하여 고급 모니터링을 구성합니다.

> **참고**: 이 실습에서는 섹션 2.3에서 `--enable-addons monitoring` 옵션으로 기본 모니터링을 활성화했습니다. 여기서는 Prometheus와 Grafana를 추가로 활성화합니다.

In [46]:
import subprocess
import warnings

# Azure CLI 확장의 pkg_resources 경고 메시지 숨기기
warnings.filterwarnings('ignore', message='.*pkg_resources.*')

print("🔧 관리형 Prometheus 및 Grafana 활성화 중...")
print()

# Azure Monitor 워크스페이스 생성
print("1️⃣ Azure Monitor 워크스페이스 생성")
MONITOR_WORKSPACE_NAME = f"{AKS_CLUSTER_NAME}-monitor"

result = subprocess.run([
    "az", "monitor", "account", "create",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--location", "koreacentral"
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Azure Monitor 워크스페이스 생성 완료: {MONITOR_WORKSPACE_NAME}")
else:
    print("⚠️ 워크스페이스가 이미 존재하거나 생성 중 오류 발생")
    print(result.stderr)

print()

# Monitor 워크스페이스 ID 가져오기
print("2️⃣ 워크스페이스 ID 확인")
ws_result = subprocess.run([
    "az", "monitor", "account", "show",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "id",
    "-o", "tsv"
], capture_output=True, text=True)

if ws_result.returncode == 0:
    MONITOR_WORKSPACE_ID = ws_result.stdout.strip()
    print(f"✅ 워크스페이스 ID: {MONITOR_WORKSPACE_ID}")
else:
    print("❌ 워크스페이스 ID 조회 실패")
    print(ws_result.stderr)

print()

# AKS에 Prometheus 애드온 활성화
print("3️⃣ AKS에 Prometheus 애드온 활성화")
result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--enable-azure-monitor-metrics",
    "--azure-monitor-workspace-resource-id", MONITOR_WORKSPACE_ID
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Prometheus 애드온 활성화 완료")
else:
    print("⚠️ Prometheus 애드온 활성화 중 오류")
    print(result.stderr)

print()

# Grafana 워크스페이스 생성
print("4️⃣ 관리형 Grafana 워크스페이스 생성")
# Grafana 이름은 2-23자 사이여야 함
GRAFANA_NAME = "aks-mini-labs-grafana"

result = subprocess.run([
    "az", "grafana", "create",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Grafana 워크스페이스 생성 완료: {GRAFANA_NAME}")
else:
    print("⚠️ Grafana가 이미 존재하거나 생성 중 오류 발생")
    print(result.stderr)

print()

# Grafana와 Prometheus 연결
print("5️⃣ Grafana와 Prometheus 데이터 소스 연결")
grafana_result = subprocess.run([
    "az", "grafana", "show",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "id",
    "-o", "tsv"
], capture_output=True, text=True)

if grafana_result.returncode == 0:
    GRAFANA_ID = grafana_result.stdout.strip()
    
    # Prometheus를 Grafana 데이터 소스로 추가
    subprocess.run([
        "az", "aks", "update",
        "--resource-group", RESOURCE_GROUP,
        "--name", AKS_CLUSTER_NAME,
        "--grafana-resource-id", GRAFANA_ID
    ])
    
    print("✅ Grafana와 Prometheus 연결 완료")
else:
    print("⚠️ Grafana 연결 중 오류")

print()
print("=" * 80)
print("🎉 모니터링 스택 설정 완료!")
print("=" * 80)
print()
print("📊 생성된 리소스:")
print(f"   - Azure Monitor 워크스페이스: {MONITOR_WORKSPACE_NAME}")
print(f"   - Grafana 워크스페이스: {GRAFANA_NAME}")
print()
print("💡 다음 단계:")
print("   1. Azure Portal에서 Grafana 워크스페이스 접속")
print("   2. Prometheus 데이터 소스 확인")
print("   3. Kubernetes 표준 대시보드 활용")

🔧 관리형 Prometheus 및 Grafana 활성화 중...

1️⃣ Azure Monitor 워크스페이스 생성
✅ Azure Monitor 워크스페이스 생성 완료: aks-mini-labs-cluster-monitor

2️⃣ 워크스페이스 ID 확인
✅ 워크스페이스 ID: /subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourcegroups/myresourcegroup/providers/microsoft.monitor/accounts/aks-mini-labs-cluster-monitor

3️⃣ AKS에 Prometheus 애드온 활성화
✅ Prometheus 애드온 활성화 완료

4️⃣ 관리형 Grafana 워크스페이스 생성
✅ Grafana 워크스페이스 생성 완료: aks-mini-labs-grafana

5️⃣ Grafana와 Prometheus 데이터 소스 연결


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


{
  "aadProfile": null,
  "addonProfiles": {
    "azurepolicy": {
      "config": null,
      "enabled": true,
      "identity": {
        "clientId": "fe5e4df5-1c69-4567-90ba-2d89ad0b6d80",
        "objectId": "ffa53af1-d1ef-4bb9-bfca-6c5a9fa07e7b",
        "resourceId": "/subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourcegroups/MC_myResourceGroup_aks-mini-labs-cluster_koreacentral/providers/Microsoft.ManagedIdentity/userAssignedIdentities/azurepolicy-aks-mini-labs-cluster"
      }
    },
    "omsagent": {
      "config": {
        "logAnalyticsWorkspaceResourceID": "/subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/DefaultResourceGroup-SE/providers/Microsoft.OperationalInsights/workspaces/DefaultWorkspace-8627ae60-01d3-4a2d-9c33-89dea54cd4b4-SE",
        "useAADAuth": "true"
      },
      "enabled": true,
      "identity": null
    }
  },
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "capacityReservationGroupId": null,
      "count":

### 5.3 Grafana 접속 URL 확인

In [47]:
import subprocess

print("🔗 Grafana 대시보드 URL 확인:")
print()

# Grafana 이름 확인
try:
    print(f"📝 사용 중인 Grafana 이름: {GRAFANA_NAME}")
    print(f"📝 리소스 그룹: {RESOURCE_GROUP}")
    print()
except NameError:
    print("⚠️  GRAFANA_NAME 변수가 설정되지 않았습니다.")
    print("   섹션 5.2를 먼저 실행하여 Grafana를 생성하세요.")
    raise

# Grafana 존재 여부 먼저 확인
print("🔍 Grafana 워크스페이스 존재 확인 중...")
check_result = subprocess.run([
    "az", "grafana", "list",
    "--resource-group", RESOURCE_GROUP,
    "--query", "[?name=='" + GRAFANA_NAME + "'].name",
    "-o", "tsv"
], capture_output=True, text=True)

if check_result.returncode == 0 and check_result.stdout.strip():
    print(f"✅ Grafana 워크스페이스 발견: {check_result.stdout.strip()}")
    print()
    
    # URL 조회
    result = subprocess.run([
        "az", "grafana", "show",
        "--name", GRAFANA_NAME,
        "--resource-group", RESOURCE_GROUP,
        "--query", "properties.endpoint",
        "-o", "tsv"
    ], capture_output=True, text=True)

    if result.returncode == 0 and result.stdout.strip():
        grafana_url = result.stdout.strip()
        print(f"✅ Grafana URL: {grafana_url}")
        print()
        print("💡 브라우저에서 위 URL로 접속하여 Grafana 대시보드를 확인하세요.")
        print("   Azure AD 계정으로 자동 로그인됩니다.")
    else:
        print("❌ Grafana URL 조회 실패")
        print(f"   에러: {result.stderr}")
else:
    print(f"❌ Grafana 워크스페이스를 찾을 수 없습니다: {GRAFANA_NAME}")
    print()
    print("💡 해결 방법:")
    print("   1. 섹션 5.2를 다시 실행하여 Grafana를 생성하세요.")
    print("   2. 또는 기존 Grafana 목록을 확인하세요:")
    print()
    
    # 리소스 그룹의 모든 Grafana 목록 출력
    list_result = subprocess.run([
        "az", "grafana", "list",
        "--resource-group", RESOURCE_GROUP,
        "--output", "table"
    ])

🔗 Grafana 대시보드 URL 확인:

📝 사용 중인 Grafana 이름: aks-mini-labs-grafana
📝 리소스 그룹: myResourceGroup

🔍 Grafana 워크스페이스 존재 확인 중...
✅ Grafana 워크스페이스 발견: aks-mini-labs-grafana

✅ Grafana URL: https://aks-mini-labs-grafana-h5fkgcc9d4fddzc6.sel.grafana.azure.com

💡 브라우저에서 위 URL로 접속하여 Grafana 대시보드를 확인하세요.
   Azure AD 계정으로 자동 로그인됩니다.


### 5.4 Grafana Prometheus 데이터 소스 확인 및 추가

Grafana에 Prometheus 데이터 소스가 제대로 설정되어 있는지 확인하고, 없으면 자동으로 추가합니다.

In [ ]:
import subprocess
import json

print("🔍 Grafana Prometheus 데이터 소스 확인 및 추가")
print("=" * 80)
print()

# 1. Azure Monitor 워크스페이스 정보 가져오기
print("1️⃣ Azure Monitor 워크스페이스 정보 확인:")
monitor_result = subprocess.run([
    "az", "monitor", "account", "show",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "{id:id, endpoint:metrics.prometheusQueryEndpoint}",
    "-o", "json"
], capture_output=True, text=True)

if monitor_result.returncode == 0:
    try:
        monitor_info = json.loads(monitor_result.stdout)
        monitor_workspace_id = monitor_info['id']
        prometheus_endpoint = monitor_info.get('endpoint', f"https://{MONITOR_WORKSPACE_NAME}.prometheus.monitor.azure.com")
        print(f"✅ Monitor Workspace ID: {monitor_workspace_id}")
        print(f"✅ Prometheus Endpoint: {prometheus_endpoint}")
    except:
        print("⚠️  Monitor 워크스페이스 정보 파싱 실패")
        monitor_workspace_id = None
        prometheus_endpoint = None
else:
    print("❌ Monitor 워크스페이스를 찾을 수 없습니다.")
    monitor_workspace_id = None
    prometheus_endpoint = None

print()

# 2. 현재 Grafana 데이터 소스 목록 확인
print("2️⃣ 현재 Grafana 데이터 소스 확인:")
ds_list_result = subprocess.run([
    "az", "grafana", "data-source", "list",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP,
    "-o", "json"
], capture_output=True, text=True)

prometheus_ds_exists = False
if ds_list_result.returncode == 0:
    try:
        data_sources = json.loads(ds_list_result.stdout)
        print(f"   총 {len(data_sources)}개의 데이터 소스 발견:")
        for ds in data_sources:
            print(f"   - {ds['name']} (타입: {ds['type']})")
            if "Azure Monitor managed service for Prometheus" in ds['name']:
                prometheus_ds_exists = True
                print("     ✅ Prometheus 데이터 소스 이미 존재!")
    except:
        print("   ⚠️  데이터 소스 목록 파싱 실패")
else:
    print("   ❌ 데이터 소스 목록 조회 실패")

print()

# 3. Prometheus 데이터 소스가 없으면 추가
if not prometheus_ds_exists and monitor_workspace_id and prometheus_endpoint:
    print("3️⃣ Prometheus 데이터 소스 추가 중...")
    print()
    
    # 구독 ID 추출
    subscription_id = monitor_workspace_id.split('/')[2]
    
    # 데이터 소스 정의
    datasource_def = {
        "access": "proxy",
        "jsonData": {
            "azureMonitorWorkspaceResourceId": monitor_workspace_id,
            "subscriptionId": subscription_id
        },
        "name": "Azure Monitor managed service for Prometheus",
        "type": "prometheus",
        "url": prometheus_endpoint
    }
    
    # 데이터 소스 추가
    ds_create_result = subprocess.run([
        "az", "grafana", "data-source", "create",
        "--name", GRAFANA_NAME,
        "--resource-group", RESOURCE_GROUP,
        "--definition", json.dumps(datasource_def)
    ], capture_output=True, text=True)
    
    if ds_create_result.returncode == 0:
        print("✅ Prometheus 데이터 소스 추가 완료!")
        try:
            result = json.loads(ds_create_result.stdout)
            print(f"   데이터 소스 ID: {result.get('id')}")
            print(f"   데이터 소스 이름: {result.get('name')}")
        except:
            pass
    else:
        print("❌ 데이터 소스 추가 실패:")
        print(ds_create_result.stderr)
elif prometheus_ds_exists:
    print("3️⃣ Prometheus 데이터 소스가 이미 존재합니다.")
    print("   ✅ 설정이 완료되어 있습니다!")
else:
    print("3️⃣ Monitor 워크스페이스 정보가 없어 데이터 소스를 추가할 수 없습니다.")
    print("   섹션 5.2를 먼저 실행하세요.")

print()
print("=" * 80)
print("💡 확인 방법:")
print("   1. Grafana URL로 접속 (섹션 5.3 참조)")
print("   2. 좌측 메뉴 > Connections > Data sources 확인")
print("   3. 'Azure Monitor managed service for Prometheus' 데이터 소스 확인")
print("   4. 좌측 메뉴 > Dashboards > Browse에서 Kubernetes 대시보드 확인")
print("=" * 80)

🔍 Grafana-Prometheus 연결 상태 확인

1️⃣ AKS 클러스터의 Grafana 연결 정보 확인:
⚠️  AKS에 Grafana가 연결되어 있지 않습니다.

2️⃣ 현재 Grafana 워크스페이스 ID 확인:
✅ 현재 Grafana ID: /subscriptions/8627ae60-01d3-4a2d-9c33-89dea54cd4b4/resourceGroups/myResourceGroup/providers/Microsoft.Dashboard/grafana/aks-mini-labs-grafana

3️⃣ 연결 상태 분석:
⚠️  AKS에 Grafana가 연결되어 있지 않습니다. 연결을 시작합니다...

✅ Grafana 연결 완료!

💡 연결이 완료되었습니다. 몇 분 후 Grafana에서 Prometheus 데이터를 확인할 수 있습니다.

💡 확인 방법:
   1. Grafana URL로 접속 (섹션 5.3 참조)
   2. 좌측 메뉴 > Connections > Data sources 확인
   3. 'Azure Monitor managed service for Prometheus' 데이터 소스 확인


### 5.5 Grafana 대시보드에서 Prometheus 표준 대시보드 조회하기
AKS에서 관리형 Prometheus와 Grafana를 활성화했다면, Azure Portal에서 관리형 Grafana 인스턴스에 접근할 수 있습니다.

1. **Azure Portal 접속**: [https://portal.azure.com](https://portal.azure.com) 에 로그인합니다.
2. **Grafana 리소스 검색**: 상단 검색창에 `Grafana`를 입력하고, 생성된 Managed Grafana 리소스를 선택합니다.
3. **Grafana 대시보드 열기**: 'Grafana 작업 영역'에서 'Grafana 대시보드' 버튼을 클릭하여 대시보드에 접속합니다.
4. **Prometheus 데이터 소스 확인**: 좌측 메뉴에서 'Connections > Data sources'로 이동하여 Prometheus가 연결되어 있는지 확인합니다.
5. **표준 대시보드 조회**: 좌측 메뉴에서 'Dashboards'를 클릭하고, 'Browse'에서 'Kubernetes / Compute Resources / Cluster', 'Kubernetes / Compute Resources / Namespace' 등 Prometheus 표준 대시보드를 선택해 클러스터/노드/파드/네임스페이스별 메트릭을 시각화할 수 있습니다.

> 참고: 관리형 Grafana는 Azure AD 계정으로 SSO(싱글사인온)되어 별도의 로그인 없이 접근할 수 있습니다.

## 6. Node Auto Provisioning(노드 자동 프로비저닝) 적용

Node Auto Provisioning(NAP)은 AKS에서 워크로드 수요에 따라 다양한 VM 크기의 노드 풀을 자동으로 생성/확장/축소하는 기능입니다. 비용 최적화와 리소스 효율성을 위해 권장되는 Azure 기능입니다.

### 6.1 사전 준비

NAP를 사용하기 위해서는 먼저 필요한 Azure CLI 확장을 설치하고 기능 플래그를 등록해야 합니다.

In [49]:
import subprocess

print("📦 aks-preview 확장 설치/업데이트 중...")
print()

# aks-preview 확장 설치 또는 업데이트
result = subprocess.run([
    "az", "extension", "add",
    "--name", "aks-preview",
    "--yes"
], capture_output=True, text=True)

if "already installed" in result.stderr.lower() or result.returncode == 0:
    print("✅ aks-preview 확장이 설치되어 있습니다. 최신 버전으로 업데이트 중...")
    subprocess.run([
        "az", "extension", "update",
        "--name", "aks-preview"
    ])
else:
    print("✅ aks-preview 확장 설치 완료")

print()
print("🔧 NodeAutoProvisioningPreview Feature Flag 등록 중...")
print()

# Feature Flag 등록
subprocess.run([
    "az", "feature", "register",
    "--namespace", "Microsoft.ContainerService",
    "--name", "NodeAutoProvisioningPreview"
])

print()
print("📋 Feature Flag 등록 상태 확인:")
subprocess.run([
    "az", "feature", "show",
    "--namespace", "Microsoft.ContainerService",
    "--name", "NodeAutoProvisioningPreview",
    "--output", "table"
])

print()
print("💡 참고:")
print("   - Feature Flag가 'Registered' 상태가 될 때까지 몇 분이 걸릴 수 있습니다.")
print("   - 'Registered' 상태 확인 후 다음 셀을 실행하세요.")

📦 aks-preview 확장 설치/업데이트 중...

✅ aks-preview 확장이 설치되어 있습니다. 최신 버전으로 업데이트 중...
✅ aks-preview 확장이 설치되어 있습니다. 최신 버전으로 업데이트 중...


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version

Use --debug for more information

Use --debug for more information



🔧 NodeAutoProvisioningPreview Feature Flag 등록 중...



/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
ERROR: (FeatureNotFound) The feature 'NodeAutoProvisioningPreview' could not be found.
Code: FeatureNotFound
Message: The feature 'NodeAutoProvisioningPreview' could not be found.
ERROR: (FeatureNotFound) The feature 'NodeAutoProvisioningPreview' could not be found.
Code: FeatureNotFound
Message: The feature 'NodeAutoProvisioningPreview' could not be found.



📋 Feature Flag 등록 상태 확인:


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version



💡 참고:
   - Feature Flag가 'Registered' 상태가 될 때까지 몇 분이 걸릴 수 있습니다.
   - 'Registered' 상태 확인 후 다음 셀을 실행하세요.


ERROR: (FeatureNotFound) The feature 'NodeAutoProvisioningPreview' could not be found.
Code: FeatureNotFound
Message: The feature 'NodeAutoProvisioningPreview' could not be found.


### 6.2 Provider 재등록

Feature Flag가 'Registered' 상태가 되면 Provider를 재등록해야 합니다.

In [50]:
import subprocess

print("🔄 Microsoft.ContainerService Provider 재등록 중...")
print()

result = subprocess.run([
    "az", "provider", "register",
    "--namespace", "Microsoft.ContainerService"
])

if result.returncode == 0:
    print("✅ Provider 재등록 완료")
    print()
    print("💡 다음 단계:")
    print("   몇 분 후 섹션 6.3을 실행하여 NAP를 활성화하세요.")
else:
    print("❌ Provider 재등록 실패")

🔄 Microsoft.ContainerService Provider 재등록 중...



/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


✅ Provider 재등록 완료

💡 다음 단계:
   몇 분 후 섹션 6.3을 실행하여 NAP를 활성화하세요.


### 6.3 기존 클러스터에 NAP 활성화

기존 AKS 클러스터에 Node Auto Provisioning을 활성화합니다.

> **중요**: NAP를 활성화하면 기존 수동 노드 풀은 유지되지만, 향후 자동 프로비저닝된 노드 풀이 추가로 생성됩니다.

In [51]:
import subprocess

print(f"🚀 기존 클러스터에 NAP 활성화 중: {AKS_CLUSTER_NAME}")
print("⏱️  예상 소요 시간: 5-10분")
print()

# NAP 활성화
result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--node-provisioning-mode", "Auto"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ NAP 활성화 완료!")
    print()
    print("📊 클러스터 상태 확인:")
    subprocess.run([
        "az", "aks", "show",
        "--resource-group", RESOURCE_GROUP,
        "--name", AKS_CLUSTER_NAME,
        "--query", "{Name:name, NodeProvisioningMode:nodeProvisioningProfile.mode}",
        "--output", "table"
    ])
else:
    print("❌ NAP 활성화 실패")
    print(result.stderr)

print()
print("💡 NAP 동작 방식:")
print("   - Pod가 스케줄될 수 없는 상황에서 AKS가 자동으로 적절한 VM 크기의 노드 풀을 생성")
print("   - 워크로드가 줄어들면 불필요한 노드 풀이 자동으로 축소/삭제")
print("   - Spot VM, GPU VM 등 다양한 VM 옵션을 자동으로 활용")

🚀 기존 클러스터에 NAP 활성화 중: aks-mini-labs-cluster
⏱️  예상 소요 시간: 5-10분

✅ NAP 활성화 완료!

📊 클러스터 상태 확인:
✅ NAP 활성화 완료!

📊 클러스터 상태 확인:


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name                   NodeProvisioningMode
---------------------  ----------------------
aks-mini-labs-cluster  Auto

💡 NAP 동작 방식:
   - Pod가 스케줄될 수 없는 상황에서 AKS가 자동으로 적절한 VM 크기의 노드 풀을 생성
   - 워크로드가 줄어들면 불필요한 노드 풀이 자동으로 축소/삭제
   - Spot VM, GPU VM 등 다양한 VM 옵션을 자동으로 활용


### 6.4 NAP 테스트: 워크로드 증가

NAP가 정상적으로 동작하는지 테스트하기 위해 Pod 수를 대폭 늘려 노드 부족 상황을 유도합니다.

> **참고**: HPA(Horizontal Pod Autoscaler)가 설정되어 있으면 수동 스케일링을 제한할 수 있으므로, 이 셀에서는 HPA를 먼저 삭제합니다.

In [61]:
import subprocess

print("🔥 워크로드 대폭 증가 - NAP 테스트 시작")
print()

# HPA가 있으면 삭제 (수동 스케일링을 위해)
print("1️⃣ HPA 확인 및 삭제:")
hpa_check = subprocess.run([
    "kubectl", "get", "hpa", "myapp"
], capture_output=True, text=True)

if hpa_check.returncode == 0:
    print("   HPA가 존재합니다. NAP 테스트를 위해 삭제합니다...")
    subprocess.run(["kubectl", "delete", "hpa", "myapp"])
    print("   ✅ HPA 삭제 완료")
else:
    print("   ✅ HPA가 없습니다. 계속 진행합니다.")

print()

# 현재 노드 상태 확인
print("2️⃣ 현재 노드 상태:")
subprocess.run(["kubectl", "get", "nodes"])

print()
print("3️⃣ 현재 노드 풀 목록:")
subprocess.run([
    "az", "aks", "nodepool", "list",
    "--resource-group", RESOURCE_GROUP,
    "--cluster-name", AKS_CLUSTER_NAME,
    "--query", "[].{Name:name, Count:count, Mode:mode}",
    "--output", "table"
])

print()
print("4️⃣ Pod 수를 50개로 증가 (NAP 트리거)...")
subprocess.run([
    "kubectl", "scale", "deployment", "myapp",
    "--replicas=100"
])

print()
print("⏱️  잠시 대기 중 (Pod 생성 대기)...")
subprocess.run(["sleep", "5"])

print()
print("5️⃣ Pod 상태 확인:")
subprocess.run(["kubectl", "get", "pods", "-o", "wide"])

print()
print("6️⃣ Pending 상태 Pod 확인:")
pending_result = subprocess.run([
    "kubectl", "get", "pods",
    "--field-selector=status.phase=Pending"
], capture_output=True, text=True)

if "No resources found" not in pending_result.stdout:
    print(pending_result.stdout)
    print("⚠️  Pending 상태의 Pod가 있습니다!")
    print("   NAP가 자동으로 새 노드를 추가할 것입니다.")
else:
    print("   ✅ 모든 Pod가 Running 상태입니다.")
    print("   현재 노드 리소스가 충분합니다.")

print()
print("=" * 80)
print("💡 다음 단계:")
print("   1. 몇 분 후 섹션 6.5를 실행하여 NAP가 새 노드 풀을 생성했는지 확인")
print("   2. Pending 상태의 Pod가 있으면 NAP가 자동으로 노드를 추가합니다")
print("   3. NAP가 트리거되지 않았다면 현재 노드 용량이 충분한 것입니다")
print("=" * 80)

🔥 워크로드 대폭 증가 - NAP 테스트 시작

1️⃣ HPA 확인 및 삭제:
   ✅ HPA가 없습니다. 계속 진행합니다.

2️⃣ 현재 노드 상태:
NAME                                STATUS   ROLES    AGE     VERSION
aks-nodepool1-39887763-vmss000000   Ready    <none>   3h12m   v1.32.7
aks-nodepool1-39887763-vmss000001   Ready    <none>   3h12m   v1.32.7

3️⃣ 현재 노드 풀 목록:
   ✅ HPA가 없습니다. 계속 진행합니다.

2️⃣ 현재 노드 상태:
NAME                                STATUS   ROLES    AGE     VERSION
aks-nodepool1-39887763-vmss000000   Ready    <none>   3h12m   v1.32.7
aks-nodepool1-39887763-vmss000001   Ready    <none>   3h12m   v1.32.7

3️⃣ 현재 노드 풀 목록:


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name       Count    Mode
---------  -------  ------
nodepool1  2        System

4️⃣ Pod 수를 50개로 증가 (NAP 트리거)...
deployment.apps/myapp scaled

⏱️  잠시 대기 중 (Pod 생성 대기)...

5️⃣ Pod 상태 확인:
NAME                     READY   STATUS              RESTARTS   AGE     IP             NODE                                NOMINATED NODE   READINESS GATES
myapp-766c44fd57-2652p   1/1     Running             0          5s      10.244.0.205   aks-nodepool1-39887763-vmss000000   <none>           <none>
myapp-766c44fd57-26l6d   1/1     Running             0          5m10s   10.244.1.127   aks-nodepool1-39887763-vmss000001   <none>           <none>
myapp-766c44fd57-27vv4   1/1     Running             0          3m4s    10.244.0.109   aks-nodepool1-39887763-vmss000000   <none>           <none>
myapp-766c44fd57-2jk2m   1/1     Running             0          5m10s   10.244.1.56    aks-nodepool1-39887763-vmss000001   <none>           <none>
myapp-766c44fd57-2rthq   1/1     Running             0          4s     

### 6.5 NAP 동작 확인

NAP에 의해 자동으로 생성된 노드 풀과 노드를 확인합니다.

In [63]:
import subprocess

print("🔍 NAP 동작 확인 - 자동 생성된 노드 풀 및 노드 조회")
print()

print("📊 현재 노드 풀 목록 (NAP에 의해 생성된 풀 포함):")
subprocess.run([
    "az", "aks", "nodepool", "list",
    "--resource-group", RESOURCE_GROUP,
    "--cluster-name", AKS_CLUSTER_NAME,
    "--output", "table"
])

print()
print("📊 현재 노드 상태:")
subprocess.run(["kubectl", "get", "nodes", "-o", "wide"])

print()
print("📊 Pod 분산 상태:")
subprocess.run([
    "kubectl", "get", "pods",
    "-o", "custom-columns=NAME:.metadata.name,STATUS:.status.phase,NODE:.spec.nodeName"
])

print()
print("💡 확인 사항:")
print("   ✅ 새로운 노드 풀이 자동 생성되었는지 확인")
print("   ✅ Pending 상태였던 Pod들이 새 노드에 스케줄링되었는지 확인")
print("   ✅ NAP가 워크로드에 맞는 적절한 VM 크기를 선택했는지 확인")

🔍 NAP 동작 확인 - 자동 생성된 노드 풀 및 노드 조회

📊 현재 노드 풀 목록 (NAP에 의해 생성된 풀 포함):


/Users/junwoojeong/.azure/cliextensions/ai-examples/azext_ai_examples/custom.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Name       OsType    VmSize           Count    MaxPods    ProvisioningState    Mode
---------  --------  ---------------  -------  ---------  -------------------  ------
nodepool1  Linux     Standard_D8a_v4  1        250        Succeeded            System

📊 현재 노드 상태:
NAME                                STATUS   ROLES    AGE     VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
aks-default-k72dc                   Ready    <none>   54s     v1.32.7   10.224.0.8    <none>        Ubuntu 22.04.5 LTS   5.15.0-1092-azure   containerd://1.7.28-1
aks-default-zm22l                   Ready    <none>   51s     v1.32.7   10.224.0.6    <none>        Ubuntu 22.04.5 LTS   5.15.0-1092-azure   containerd://1.7.28-1
aks-nodepool1-39887763-vmss000000   Ready    <none>   3h16m   v1.32.7   10.224.0.5    <none>        Ubuntu 22.04.5 LTS   5.15.0-1092-azure   containerd://1.7.28-1
aks-system-surge-zgd7b              Ready    <none>   83s     v1.32.7   10.224.0.7 

### 6.6 NAP 테스트: 워크로드 감소

워크로드를 줄여서 NAP가 자동으로 노드 풀을 축소/삭제하는지 확인합니다.

In [64]:
import subprocess

print("📉 워크로드 감소 - NAP 자동 축소 테스트")
print()

print("🔄 Pod 수를 다시 2개로 축소...")
subprocess.run([
    "kubectl", "scale", "deployment", "myapp",
    "--replicas=2"
])

print()
print("📊 현재 Pod 상태:")
subprocess.run(["kubectl", "get", "pods"])

print()
print("💡 NAP 자동 축소 확인 방법:")
print("   1. 몇 분 후 다시 노드 풀 목록을 확인")
print("   2. 불필요한 노드 풀이 자동으로 삭제되는지 모니터링")
print()
print("   실시간 모니터링 명령:")
print("   kubectl get nodes --watch")
print("   az aks nodepool list --resource-group", RESOURCE_GROUP, "--cluster-name", AKS_CLUSTER_NAME)

📉 워크로드 감소 - NAP 자동 축소 테스트

🔄 Pod 수를 다시 2개로 축소...
deployment.apps/myapp scaled

📊 현재 Pod 상태:
NAME                     READY   STATUS    RESTARTS   AGE
myapp-766c44fd57-2652p   1/1     Running   0          4m58s
myapp-766c44fd57-27vv4   1/1     Running   0          7m57s
myapp-766c44fd57-2rthq   1/1     Running   0          4m57s
myapp-766c44fd57-48ct6   1/1     Running   0          7m57s
myapp-766c44fd57-4cm5x   1/1     Running   0          4m58s
myapp-766c44fd57-4mz62   1/1     Running   0          2m40s
myapp-766c44fd57-4plh4   1/1     Running   0          4m57s
myapp-766c44fd57-4s4j8   1/1     Running   0          3m37s
myapp-766c44fd57-4spbn   1/1     Running   0          10m
myapp-766c44fd57-4ss6l   1/1     Running   0          4m57s
myapp-766c44fd57-4zrlg   1/1     Running   0          7m57s
myapp-766c44fd57-5bv6q   1/1     Running   0          4m57s
myapp-766c44fd57-5pqrg   1/1     Running   0          3m42s
myapp-766c44fd57-5q8hf   1/1     Running   0          4m57s
myapp-766c44

### 6.7 NAP 장점 정리

Node Auto Provisioning을 사용하면 다음과 같은 이점을 얻을 수 있습니다:

- ✅ **비용 최적화**: 워크로드에 따라 자동으로 스케일링되어 불필요한 리소스 비용 절감
- ✅ **리소스 효율성**: 다양한 VM 크기를 자동으로 선택하여 최적의 리소스 활용
- ✅ **운영 자동화**: 수동 노드 풀 관리 없이 자동으로 확장/축소
- ✅ **유연성**: GPU, Spot VM 등 다양한 VM 옵션 자동 활용
- ✅ **고가용성**: Pod가 Pending 상태에 머무르지 않고 즉시 스케줄링

> **참고**: 자세한 내용은 [Azure 공식 문서](https://learn.microsoft.com/en-us/azure/aks/node-autoprovision?tabs=azure-cli)를 참조하세요.

## 7. 정리

AKS 실습을 통해 클러스터 생성, 모니터링, 오토스케일링, Node Auto Provisioning 등 Kubernetes 기반 애플리케이션의 배포와 운영 자동화 전 과정을 GitHub Codespaces 환경에서 단계별로 실습했습니다.

### 실습한 내용 요약
- ✅ AKS 클러스터 생성 및 구성
- ✅ ACR 연동 및 이미지 배포
- ✅ LoadBalancer Service를 통한 외부 노출
- ✅ 수동 및 자동 스케일링 (HPA)
- ✅ Azure Monitor, Prometheus, Grafana를 통한 모니터링
- ✅ Node Auto Provisioning 테스트

> **다음 단계**: `03-cicd-automation.ipynb`에서 GitHub Actions와 Azure Pipelines를 사용한 CI/CD 자동화를 실습합니다.